<a href="https://colab.research.google.com/github/sarabdar/keras/blob/main/Kera_Tunner_Hyper_Parameter_Tuning.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [1]:
import sys

# Increase recursion limit to prevent potential issues
sys.setrecursionlimit(100000)

In [3]:
!pip install keras-tuner

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 129.4/129.4 kB 4.0 MB/s eta 0:00:00


In [4]:
# Step 2: Import necessary libraries
import keras_tuner as kt
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import Dense, Flatten
from tensorflow.keras.datasets import mnist
from tensorflow.keras.optimizers import Adam
import os
import warnings

# Suppress all Python warnings
warnings.filterwarnings('ignore')

# Set TensorFlow log level to suppress warnings and info messages
os.environ['TF_CPP_MIN_LOG_LEVEL'] = '2'  # 0 = all logs, 1 = filter out INFO, 2 = filter out INFO and WARNING, 3 = ERROR only

# Using mnist dataset

In [7]:
!pip install scikit-learn

In [8]:
from tensorflow import keras
from sklearn.model_selection import train_test_split

# Load the MNIST dataset
(x_all, y_all), _ = keras.datasets.mnist.load_data()

# Flatten and normalize the images
x_all = x_all.reshape((x_all.shape[0], -1)).astype("float32") / 255.0

# Split into train+val and test (80/20)
x_temp, x_test, y_temp, y_test = train_test_split(x_all, y_all, test_size=0.2, random_state=42)

# Split train+val into train and validation (75/25 of 80% = 60/20 overall)
x_train, x_val, y_train, y_val = train_test_split(x_temp, y_temp, test_size=0.25, random_state=42)

In [6]:

# Step 3: Load and preprocess the MNIST dataset
(x_train, y_train), (x_val, y_val) = mnist.load_data()
x_train, x_val = x_train / 255.0, x_val / 255.0

print(f'Training data shape: {x_train.shape}')
print(f'Validation data shape: {x_val.shape}')

Training data shape: (60000, 28, 28)
Validation data shape: (10000, 28, 28)


**Define a model-building function:**
- Create a function `build_model` that takes a `HyperParameters` object as input.
- Use the `HyperParameters` object to define the number of units in a dense layer and the learning rate for the optimizer.
- Compile the model with sparse categorical cross-entropy loss and Adam optimizer.

In [13]:
# Define a model-building function

def build_model(hp):
    model = Sequential([
        Flatten(input_shape=(784,)),
        Dense(units=hp.Int('units', min_value=32, max_value=512, step=32), activation='relu'),
        Dense(10, activation='softmax')
    ])

    model.compile(
        optimizer=Adam(learning_rate=hp.Float('learning_rate', min_value=1e-4, max_value=1e-2, sampling='LOG')),
        loss='sparse_categorical_crossentropy',
        metrics=['accuracy']
    )

    return model

# Main tuners in Keras Tuner:

- RandomSearch

- Hyperband

- BayesianOptimization

### Random Search Tuner

Class: kt.RandomSearch

#### What it does:

- Randomly tries combinations of hyperparameters

- Simple and easy to use

#### Good when:

- Search space is small or medium

- You want something quick and simple

## Hyperband Tuner

Class: kt.Hyperband

#### What it does:

- Uses a smart strategy to stop bad models early

- Trains many models for few epochs, and only continues the best ones

#### Good when:

- Training is expensive

- You want efficient use of time/resources

# Bayesian Optimization Tuner

Class: kt.BayesianOptimization

#### What it does:

- Uses probability to predict good hyperparameters

- Learns from previous trials

#### Good when:

- Search space is large

- You want smarter searching instead of random guessing

In [15]:
# Create a RandomSearch Tuner

tuner = kt.Hyperband(
    build_model,
    objective='val_accuracy',
    executions_per_trial=2,
    directory='my_dir',
    project_name='intro_to_kt'
)

# Display a summary of the search space
tuner.search_space_summary()

Reloading Tuner from my_dir/intro_to_kt/tuner0.json
Search space summary
Default search space size: 2
units (Int)
{'default': None, 'conditions': [], 'min_value': 32, 'max_value': 512, 'step': 32, 'sampling': 'linear'}
learning_rate (Float)
{'default': 0.0001, 'conditions': [], 'min_value': 0.0001, 'max_value': 0.01, 'step': None, 'sampling': 'log'}


In [16]:
# Run the hyperparameter search
tuner.search(x_train, y_train, epochs=5, validation_data=(x_val, y_val))

# Display a summary of the results
tuner.results_summary()

Trial 200 Complete [00h 06m 06s]
val_accuracy: 0.9802083373069763

Best val_accuracy So Far: 0.9811666905879974
Total elapsed time: 03h 49m 39s

Search: Running Trial #201

Value             |Best Value So Far |Hyperparameter
352               |512               |units
0.00097278        |0.00056682        |learning_rate
100               |34                |tuner/epochs
34                |12                |tuner/initial_epoch
3                 |4                 |tuner/bracket
3                 |3                 |tuner/round
0195              |0130              |tuner/trial_id

Epoch 35/100
1125/1125 ━━━━━━━━━━━━━━━━━━━━ 10s 8ms/step - accuracy: 0.9988 - loss: 0.0034 - val_accuracy: 0.9760 - val_loss: 0.1458
Epoch 36/100
1125/1125 ━━━━━━━━━━━━━━━━━━━━ 9s 8ms/step - accuracy: 0.9993 - loss: 0.0027 - val_accuracy: 0.9783 - val_loss: 0.1299
Epoch 37/100
1125/1125 ━━━━━━━━━━━━━━━━━━━━ 9s 8ms/step - accuracy: 0.9992 - loss: 0.0030 - val_accuracy: 0.9759 - val_loss: 0.1433
Epoch 38/100
112

KeyboardInterrupt: 

### Ok i will be stopping it now at it has been like 4 hours now

In [17]:
# Step 1: Retrieve the best hyperparameters

best_hps = tuner.get_best_hyperparameters(num_trials=1)[0]
print(f"""

The optimal number of units in the first dense layer is {best_hps.get('units')}.

The optimal learning rate for the optimizer is {best_hps.get('learning_rate')}.

""")

# Step 2: Build and Train the Model with Best Hyperparameters
model = tuner.hypermodel.build(best_hps)
model.fit(x_train, y_train, epochs=10, validation_split=0.2)

# Evaluate the model on the test set
test_loss, test_acc = model.evaluate(x_val, y_val)
print(f'Test accuracy: {test_acc}')

 

The optimal number of units in the first dense layer is 512. 

The optimal learning rate for the optimizer is 0.0005668181079162232. 


Epoch 1/10
900/900 ━━━━━━━━━━━━━━━━━━━━ 10s 10ms/step - accuracy: 0.8517 - loss: 0.5356 - val_accuracy: 0.9390 - val_loss: 0.2054
Epoch 2/10
900/900 ━━━━━━━━━━━━━━━━━━━━ 10s 10ms/step - accuracy: 0.9560 - loss: 0.1500 - val_accuracy: 0.9567 - val_loss: 0.1457
Epoch 3/10
900/900 ━━━━━━━━━━━━━━━━━━━━ 9s 10ms/step - accuracy: 0.9728 - loss: 0.0931 - val_accuracy: 0.9608 - val_loss: 0.1232
Epoch 4/10
900/900 ━━━━━━━━━━━━━━━━━━━━ 8s 9ms/step - accuracy: 0.9803 - loss: 0.0677 - val_accuracy: 0.9665 - val_loss: 0.1096
Epoch 5/10
900/900 ━━━━━━━━━━━━━━━━━━━━ 9s 10ms/step - accuracy: 0.9884 - loss: 0.0446 - val_accuracy: 0.9665 - val_loss: 0.1115
Epoch 6/10
900/900 ━━━━━━━━━━━━━━━━━━━━ 8s 9ms/step - accuracy: 0.9932 - loss: 0.0296 - val_accuracy: 0.9693 - val_loss: 0.1020
Epoch 7/10
900/900 ━━━━━━━━━━━━━━━━━━━━ 9s 10ms/step - accuracy: 0.9939 - loss: 0.0247 